In [211]:
import random
import tensorflow as tf
import numpy as np

In [212]:
symbol = {
    '+':10000,
    '-':20000,
    '*':30000,
    '/':40000,
}

symbol_s = {
    10000:'+',
    20000:'-',
    30000:'*',
    40000:'/',
}

In [213]:
def convertToArray(s):
    a = []
    prev_index=0
    for i in range(len(s)):
        if s[i] in symbol.keys():
            a.append(int(s[prev_index:i]))
            a.append(symbol[s[i]])
            prev_index = i+1
            
    a.append(int(s[prev_index:len(s)]))
    return a
    
#convert equation to array

def convertToString(a):
    s=""
    for i in a:
        if i in symbol_s.keys():
            s=s+(symbol_s[i])
        else:
            s=s+(str(i))
    return s

In [214]:
ss = convertToArray("11+2/344*4")
print(ss)
print(convertToString(ss))

[11, 10000, 2, 40000, 344, 30000, 4]
11+2/344*4


In [237]:
def generateEquation():
    eq = []
    eq.append(random.randint(lower_bound, upper_bound))
    for i in range(4):
        eq.append(random.randint(1, 4)*1000)
        eq.append(random.randint(lower_bound, upper_bound))
    return eq

In [238]:
test = generateEquation()
print(convertToString(test))

730009400042000330008


In [239]:
def generateStep(a):  
    mark=[[],[]] # step1 and step 2 marks
    sol = [a]
    
    n = len(a)//2
    
    prev = a
    for i in range(n-1,-1,-1):
        x = random.randint(0,i)
        #print("i="+str(i)+"x="+str(x))
        #print(prev[:2*x],prev[2*x+3:])
        
        step = prev[:2*x]
        
        #process step
        sym = a[2*x+1]
        #print(sym)
        error = random.randint(0,4)//4
        if sym == 1000:
            step.append(prev[2*x] + prev[2*x+2]+error)
            mark[0].append(0) if (3000 in prev) or (4000 in prev) else mark[0].append(1)
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 2000:
            step.append(prev[2*x] - prev[2*x+2]+error)
            mark[0].append(0) if (3000 in prev) or (4000 in prev) else mark[0].append(1)
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 3000:
            step.append(prev[2*x] * prev[2*x+2]+error)
            mark[0].append(1);
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 4000:
            step.append(prev[2*x] / prev[2*x+2]+error)
            mark[0].append(1);
            mark[1].append(0) if error != 0 else mark[1].append(1)
        
        step += prev[2*x+3:]
        step += [0]*(len(a)-len(step))
        #print(step)
        sol.append(step)
        prev = step
    
    
    
    
    
    return (sol,mark)

In [240]:
test = generateEquation()
for step in generateStep(test):
    for i in step:
        print(i)

[10, 2000, 6, 1000, 8, 3000, 10, 2000, 4]
[4, 1000, 8, 3000, 10, 2000, 4, 0, 0]
[4, 1000, 18, 2000, 4, 0, 0, 0, 0]
[4, 1000, 23, 0, 0, 0, 0, 0, 0]
[-19, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1]
[1, 1, 0, 1]


In [244]:
upper_bound=10
lower_bound=1
size = 5000

traindata=[]
trainlabel1=[]
trainlabel2=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            traindata.append(sol[j:j+2])
        trainlabel1+=mark[0]
        trainlabel2+=mark[1]
    except:
        i=i-1

In [245]:
model1 = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(18, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model1.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [246]:
model1.fit(traindata, trainlabel1, epochs=20)

Epoch 1/20
622/622 [==============================] - 0s 483us/step - loss: 34.5285 - accuracy: 0.6938
Epoch 2/20
622/622 [==============================] - 0s 475us/step - loss: 9.3976 - accuracy: 0.6986
Epoch 3/20
622/622 [==============================] - 0s 481us/step - loss: 6.3256 - accuracy: 0.7069
Epoch 4/20
622/622 [==============================] - 0s 475us/step - loss: 5.0104 - accuracy: 0.7147
Epoch 5/20
622/622 [==============================] - 0s 471us/step - loss: 4.3968 - accuracy: 0.7239
Epoch 6/20
622/622 [==============================] - 0s 492us/step - loss: 3.7681 - accuracy: 0.7268
Epoch 7/20
622/622 [==============================] - 0s 476us/step - loss: 3.2996 - accuracy: 0.7295
Epoch 8/20
622/622 [==============================] - 0s 476us/step - loss: 2.8477 - accuracy: 0.7383
Epoch 9/20
622/622 [==============================] - 0s 510us/step - loss: 2.6370 - accuracy: 0.7325
Epoch 10/20
622/622 [==============================] - 0s 478us/step - loss: 2.28

In [222]:
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(18, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model2.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [223]:
model2.fit(traindata, trainlabel2, epochs=15)

Epoch 1/15
622/622 [==============================] - 0s 460us/step - loss: 2250.4890 - accuracy: 0.6332
Epoch 2/15
622/622 [==============================] - 0s 467us/step - loss: 91.4747 - accuracy: 0.6767
Epoch 3/15
622/622 [==============================] - 0s 467us/step - loss: 37.0919 - accuracy: 0.6815
Epoch 4/15
622/622 [==============================] - 0s 457us/step - loss: 25.0243 - accuracy: 0.6850
Epoch 5/15
622/622 [==============================] - 0s 459us/step - loss: 25.2566 - accuracy: 0.6772
Epoch 6/15
622/622 [==============================] - 0s 470us/step - loss: 24.7202 - accuracy: 0.6783
Epoch 7/15
622/622 [==============================] - 0s 459us/step - loss: 23.3125 - accuracy: 0.6775
Epoch 8/15
622/622 [==============================] - 0s 463us/step - loss: 22.2033 - accuracy: 0.6770
Epoch 9/15
622/622 [==============================] - 0s 460us/step - loss: 21.6818 - accuracy: 0.6817
Epoch 10/15
622/622 [==============================] - 0s 462us/step - 

In [247]:
test_size = 10

testdata=[]
testlabel1=[]
testlabel2=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            testdata.append(sol[j:j+2])
        testlabel1+=mark[0]
        testlabel2+=mark[1]
    except:
        i=i-1

In [248]:
model1.evaluate(testdata, testlabel1)

620/620 [==============================] - 0s 386us/step - loss: 1.8216 - accuracy: 0.6988


[1.8216372728347778, 0.6988201141357422]

In [227]:
model2.evaluate(testdata, testlabel2)

620/620 [==============================] - 0s 382us/step - loss: 24.6680 - accuracy: 0.4755


[24.6679744720459, 0.4755396544933319]